# Using MLRun Projects and GIT
  --------------------------------------------------------------------

Loading or creating a full project with multiple functions and workflow and working wit Git.

#### **notebook how-to's**
* Load a project with multiple functions from Git
* Run automated workflows (using KubeFlow)
* Update, maintain and debug code 

<a id='top'></a>
#### **steps**
**[Load project from Git or Archive](#load-project)**<br>
**[Run a pipeline workflow](#run-pipeline)**<br>
**[Updating the project and code](#update-project)**<br>

In [1]:
from mlrun import load_project, code_to_function

<a id='load-project'></a>
## Load project from Git or Archive

Projects can be stored in a Git repo or in a tar archive (on object store like S3, V3IO).

`load_project(context, url)` will load/clone the project to the local context dir and build the project object from the `project.yaml` file in the git/archive root directory. 

> Note: If URL is not specified it will use the context and search for Git repo inside it, or use the `init_git=True` flag to initialize a Git repo in the target context directory.

You can also clone the code into a dir using a CLI commands:

`mlrun project my-proj/ -u git://github.com/mlrun/demo-xgb-project.git`


In [2]:
# source Git Repo, YOU SHOULD fork this to your account and use the fork
url = 'git://github.com/yjb-ds/demo-xgb-project.git'

# alternatively can use tar files, e.g.
#url = 'v3io:///users/admin/tars/src-project.tar.gz'

project_dir = './'  # change if you want to clone into a different dir
proj = load_project(project_dir, url)

In [3]:
# change dir into the project dir if you are not running from it
!cd {project_dir}

In [4]:
proj.source

'git://github.com/yjb-ds/demo-xgb-project.git#refs/heads/v0.4.6'

In [5]:
project_dir = './'  # change if you want to clone into a different dir
proj = load_project(project_dir, url)

Examine the project object, note it contains lists of `functions` and `workflows` which will be used in the project. Functions can be local to the project or referenced to (via a URL to .ipynb, .py, .yaml file and/or container image). 

In [6]:
print(proj.to_yaml())

name: iris
functions:
- url: /User/repos/demo-xgb-project/src/iris.yaml
  name: xgb
- url: /User/repos/functions/xgb_serving/xgb_serving.ipynb
  name: serving
workflows:
- name: main
  path: /User/repos/demo-xgb-project/src/workflow.py
artifacts: []



In [7]:
# You can update the function .py and .yaml from a notebook (code + spec)
# the "code_output" option will generate a .py file from our notebook which can be used for src control and local runs
xgbfn = code_to_function('xgb', 
                         filename='/User/repos/demo-xgb-project/notebooks/train-xgboost.ipynb',
                         kind='job',
                         code_output='/User/repos/demo-xgb-project/src/iris.py',
                         image="mlrun/ml-models:0.4.6-py37")
xgbfn.export('/User/repos/demo-xgb-project/src/iris.yaml')

[mlrun] 2020-04-29 21:15:37,440 function spec saved to path: /User/repos/demo-xgb-project/src/iris.yaml


<a id='update-project'></a>
## Updating the project and code

A user can update the code using the standard Git process (commit, push, ..), if you update/edit the project object you need to run `proj.save()` which will update the `project.yaml` file in your context directory, followed by pushing your updates.

You can use `proj.push(branch, commit_message, add=[])` which will do the work for you (save the yaml, commit updates, push)

> Note: you must push updates before you build functions or run workflows since the builder will pull the code from the git repo.

If you are using containerized Jupyter you may need to first set your Git parameters, e.g.:

```
!git config --global user.email "<my@email.com>"
!git config --global user.name "<name>"
!git config --global credential.helper store
```

In [ ]:
proj.push('v0.4.6', 'src-mlrun project edits')

If you want to pull changes done by others use `proj.pull()`, if you need to update the project spec (since the yaml file changed) use `proj.reload()` and for updating the local/remote function specs use `proj.sync_functions()` (or add `sync=True` to `.reload()`).

In [ ]:
proj.pull()

<a id='run-pipeline'></a>
## Run a pipeline workflow
You can check the [workflow.py](src/workflow.py) file to see how functions objects are initialized and used (by name) inside the workflow.
The `workflow.py` file has two parts, initialize the function objects and define pipeline dsl (connect the function inputs and outputs).

> Note the pipeline can include CI steps like building container images and deploying models.

### Initializing the functions (e.g. mount them on the v3io fabric)
```python
def init_functions(functions: dict, params=None, secrets=None):
    for f in functions.values():
        f.apply(mount_v3io())
        
```
<br>

### Workflow DSL:
```python
@dsl.pipeline(
    name='My XGBoost training pipeline',
    description='Shows how to use mlrun.'
)
def kfpipeline(
        eta=[0.1, 0.2, 0.3], gamma=[0.1, 0.2, 0.3]
):
    # first step build the function container
    builder = funcs['xgb'].deploy_step(with_mlrun=False)

    # use xgb.iris_generator function to generate data (container image from the builder)
    ingest = funcs['xgb'].as_step(name='ingest_iris', handler='iris_generator',
        image=builder.outputs['image'],
        outputs=['iris_dataset'])

    # use xgb.xgb_train function to train on the data (from the generator step)
    train = funcs['xgb'].as_step(name='xgb_train', handler='xgb_train',
        image=builder.outputs['image'],
        hyperparams={'eta': eta, 'gamma': gamma},
        selector='max.accuracy',
        inputs={'dataset': ingest.outputs['iris_dataset']},
        outputs=['model'])

    # deploy the trained model using a nuclio real-time function
    deploy = funcs['serving'].deploy_step(models={'iris_v1': train.outputs['model']})
```

### Run
use the `run` method to execute a workflow, you can provide alternative arguments and specify the default target for workflow artifacts.<br>
The workflow ID is returned and can be used to track the progress or you can use the hyperlinks

> Note: The same command can be issued through CLI commands:<br>
    `mlrun project my-proj/ -r main -p "v3io:///users/admin/mlrun/kfp/{{workflow.uid}}/"`

The dirty flag allow us to run a project with uncommited changes (when the notebook is in the same git dir it will always be dirty)

In [8]:
proj.run('main', 
         arguments={}, 
         artifact_path='v3io:///users/admin/artifacts/xgb_trainer/{{workflow.uid}}/', 
         dirty=True)

[mlrun] 2020-04-29 21:15:52,586 WARNING!, you seem to have uncommitted git changes, use .push()


ValueError: please specify the function kind

## Replacing the source path to speed debug

Instead of updating Git anytime we modify code we can build the code from the shared file system on the cluster (the build container will mount to the same location with the code instead of reading from Git).

We need to change the project source to point to the shared file system URL of our context directory (e.g. v3io), and we can re-run the workflow. 

In [ ]:
proj.source = 'v3io:///users/admin/repos/demo-xgb-project'

**[back to top](#top)**